# Projeto de sistema de recomendação de filmes
## Passo 1) Lendo o dataset

O dataset utilizado é o MovieLens, cujo detalhamento pode ser encontrado no link http://files.grouplens.org/datasets/movielens/ml-20m-README.html

Os arquivos são disponibilizados no formato '.csv'. Assim, faremos a leitura dos arquivos com a biblioteca Pandas.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Lendo os arquivos
ratings = pd.read_csv('ml-20m/ratings.csv')
movies = pd.read_csv('ml-20m/movies.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

In [3]:
# Mesclando os dados dos filmes com as avaliações
ratings_movies = pd.merge(ratings, movies, on='movieId').drop('timestamp', axis=1)
ratings_movies.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
# Retirando as linhas sem tag (NaN)
#tags = tags.dropna().drop('timestamp', axis=1)
#tags.head()

## Passo 2) Como explorar os dados?

Neste ponto a intenção é conhecer melhor os dados, explorando através de questões. Primeiro respondemos questões simples e mais genéricas, como os 10 filmes com mais avaliações 5 estrelas. Depois partimos para questões mais complexas relacionadas a preferência geral.

In [5]:
# Filtrar os 10 filmes com maior NÚMERO de avaliações 5 estrelas, listando-os pelo título:
top_5star_movies = ratings_movies[ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_movies

Shawshank Redemption, The (1994)             31896
Pulp Fiction (1994)                          27762
Silence of the Lambs, The (1991)             22513
Schindler's List (1993)                      22355
Star Wars: Episode IV - A New Hope (1977)    22117
Forrest Gump (1994)                          21292
Godfather, The (1972)                        20251
Usual Suspects, The (1995)                   19914
Matrix, The (1999)                           18582
Braveheart (1995)                            18467
Name: title, dtype: int64

In [6]:
# Os 25 filmes com maior MÉDIA de estrelas, listando-os pelo título:
top_meanstar = ratings_movies.groupby('title').agg({'rating': [np.size, np.mean]})
top_meanstar.sort_values([('rating', 'mean')], ascending=False).head(25)

rating     
                                                     size mean
title                                                         
Prom Queen: The Marc Hall Story (2004)                1.0  5.0
The Garden of Sinners - Chapter 5: Paradox Para...    1.0  5.0
Death of a Nation - The Timor Conspiracy (1994)       1.0  5.0
Poison (1951)                                         1.0  5.0
Sun Kissed (2012)                                     1.0  5.0
Giorgino (1994)                                       1.0  5.0
Schmatta: Rags to Riches to Rags (2009)               1.0  5.0
De la servitude moderne (2009)                        1.0  5.0
The Encounter (2010)                                  1.0  5.0
Best of Ernie and Bert, The (1988)                    1.0  5.0
Summer Wishes, Winter Dreams (1973)                   1.0  5.0
The Floating Castle (2012)                            1.0  5.0
Marihuana (1936)                                      1.0  5.0
Lady of Chance, A (1928)                              1.0  5.0
Such Hawks Such Hounds (2008)                         1.0  5.0
Tales That Witness Madness (1973)                     1.0  5.0
B-Side (2013)                                         1.0  5.0
Consuming Kids: The Commercialization of Childh...    2.0  5.0
Abendland (2011)                                      1.0  5.0
Going Down in LA-LA Land (2011)                       1.0  5.0
Mishen (Target) (2011)                                1.0  5.0
Argentina latente (2007)                              1.0  5.0
Flight of the Conchords: A Texan Odyssey (2006)       1.0  5.0
Always for Pleasure (1978)                            1.0  5.0
The Green (2011)                                      1.0  5.0

##### Diferença entre usar o número de avaliações 5 estrelas e a média de estrelas por filme:

Ao utilizar a número de avaliações 5 estrelas nós selecionamos os títulos mais populares e bem avaliados, entretanto, podem ficar subamostrados os bons títulos mas com poucas avaliações. Neste ponto supre tal necessidade o uso da média de estrelas por título, mas este último método também oferece a desvantagem de selecionar títulos que não são populares e também títulos que tiveram pouquissimas avaliações, contudo positivas.

### Questões genéricas relacionadas aos gêneros

* Quais os filmes com mais avaliações 5 estrelas dentro de cada gênero?

In [7]:
# Um exemplo para filtrar os filmes por gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama = ratings_movies[ratings_movies['genres'].str.contains('Drama')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Shawshank Redemption, The (1994)    31896
Pulp Fiction (1994)                 27762
Schindler's List (1993)             22355
Forrest Gump (1994)                 21292
Godfather, The (1972)               20251
Braveheart (1995)                   18467
American Beauty (1999)              15719
Fargo (1996)                        15232
Fight Club (1999)                   14623
Godfather: Part II, The (1974)      11737
Name: title, dtype: int64

### Questões relacionadas a preferência por gênero

Usuários avaliam de apenas um gênero, ou mais gêneros? Eles gostam desses gêneros?

Para responder a tal pergunta nós seguiremos as seguintes etapas: 

 1. Determinar o número de avaliações por gênero e usuário
 
 2. Determinar a participação de cada gênero no número de avaliações de cada usuário
 
 3. Encontrar quais os gêneros que compõe a maior parte das avaliações do usuário (determinar um threshold)
 
 4. Fazer a média do número de gêneros identificados por usuário como forma de metrificar a importância do gênero na escolha do filme.
 
O resultado mais próximo de 1 significa que os usuários avaliam em média somente um gênero; neste caso, identificar este gênero é muito importante para o sistema de recomendação. 
Já o resultado mais distante de 1 significa que os usuários avaliam em média muitos gêneros. 

In [8]:
# Determinando o número de avaliações porgênero para cada usuário

# 1. definindo todos os gêneros que existem:
genre_labels = set()
for s in ratings_movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# 2. desmembrando todos os gêneros em colunas separadas
import re
genres_df = pd.DataFrame(dict((genre, ratings_movies.genres.str.contains(genre, re.IGNORECASE))
                             for genre in genre_labels))
ratings_movies_expand = genres_df.join(ratings_movies)

# 3. contar quantas vezes os gêneros são verdadeiros por usuário
n_gen_user = ratings_movies_expand.groupby('userId').agg({genre:[np.sum] for genre in genre_labels})
n_gen_user.head()

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()


,Thriller,Romance,Adventure,Comedy,Children,Musical,(no genres listed),Action,War,Animation,Crime,Documentary,Drama,Horror,Sci-Fi,IMAX,Fantasy,Western,Film-Noir,Mystery
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
userId,,,,,,,,,,,,,,,,,,,,
1,42.0,11.0,73.0,41.0,19.0,3.0,0.0,66.0,9.0,10.0,21.0,0.0,43.0,45.0,40.0,2.0,69.0,4.0,0.0,18.0
2,19.0,6.0,17.0,10.0,1.0,2.0,0.0,19.0,4.0,1.0,1.0,0.0,19.0,18.0,23.0,1.0,1.0,2.0,1.0,4.0
3,50.0,16.0,50.0,52.0,10.0,6.0,0.0,61.0,6.0,4.0,21.0,1.0,58.0,32.0,93.0,0.0,20.0,3.0,1.0,11.0
4,13.0,4.0,6.0,11.0,4.0,2.0,0.0,13.0,1.0,2.0,6.0,0.0,8.0,0.0,5.0,0.0,3.0,1.0,0.0,3.0
5,15.0,16.0,21.0,24.0,11.0,8.0,0.0,18.0,1.0,6.0,7.0,0.0,27.0,1.0,10.0,3.0,11.0,2.0,0.0,2.0


In [9]:
#  Determinando a participação percentual de cada gênero no número de avaliações de cada usuário
n_gen_user['total'] = n_gen_user.apply(sum, axis=1)

tax_gen_user = n_gen_user.div(n_gen_user['total'], axis=0).mul(100)
tax_gen_user.head()

,Thriller,Romance,Adventure,Comedy,Children,Musical,(no genres listed),Action,War,Animation,...,Documentary,Drama,Horror,Sci-Fi,IMAX,Fantasy,Western,Film-Noir,Mystery,total
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,
userId,,,,,,,,,,,,,,,,,,,,,
1,8.139535,2.131783,14.147287,7.945736,3.682171,0.581395,0.0,12.790698,1.744186,1.937984,...,0.00000,8.333333,8.720930,7.751938,0.387597,13.372093,0.775194,0.000000,3.488372,100.0
2,12.751678,4.026846,11.409396,6.711409,0.671141,1.342282,0.0,12.751678,2.684564,0.671141,...,0.00000,12.751678,12.080537,15.436242,0.671141,0.671141,1.342282,0.671141,2.684564,100.0
3,10.101010,3.232323,10.101010,10.505051,2.020202,1.212121,0.0,12.323232,1.212121,0.808081,...,0.20202,11.717172,6.464646,18.787879,0.000000,4.040404,0.606061,0.202020,2.222222,100.0
4,15.853659,4.878049,7.317073,13.414634,4.878049,2.439024,0.0,15.853659,1.219512,2.439024,...,0.00000,9.756098,0.000000,6.097561,0.000000,3.658537,1.219512,0.000000,3.658537,100.0
5,8.196721,8.743169,11.475410,13.114754,6.010929,4.371585,0.0,9.836066,0.546448,3.278689,...,0.00000,14.754098,0.546448,5.464481,1.639344,6.010929,1.092896,0.000000,1.092896,100.0


In [10]:
# Encontrando qual gênero que compõe a maior parte das avaliações do usuário e alocando na coluna 'max'
tax_pref = tax_gen_user.drop('total', axis=1)
tax_pref['max'] = tax_pref.apply(np.argmax, axis=1)
tax_pref.head()

C:\Users\mariv\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,Thriller,Romance,Adventure,Comedy,Children,Musical,(no genres listed),Action,War,Animation,...,Documentary,Drama,Horror,Sci-Fi,IMAX,Fantasy,Western,Film-Noir,Mystery,max
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,
userId,,,,,,,,,,,,,,,,,,,,,
1,8.139535,2.131783,14.147287,7.945736,3.682171,0.581395,0.0,12.790698,1.744186,1.937984,...,0.00000,8.333333,8.720930,7.751938,0.387597,13.372093,0.775194,0.000000,3.488372,"(Adventure, sum)"
2,12.751678,4.026846,11.409396,6.711409,0.671141,1.342282,0.0,12.751678,2.684564,0.671141,...,0.00000,12.751678,12.080537,15.436242,0.671141,0.671141,1.342282,0.671141,2.684564,"(Sci-Fi, sum)"
3,10.101010,3.232323,10.101010,10.505051,2.020202,1.212121,0.0,12.323232,1.212121,0.808081,...,0.20202,11.717172,6.464646,18.787879,0.000000,4.040404,0.606061,0.202020,2.222222,"(Sci-Fi, sum)"
4,15.853659,4.878049,7.317073,13.414634,4.878049,2.439024,0.0,15.853659,1.219512,2.439024,...,0.00000,9.756098,0.000000,6.097561,0.000000,3.658537,1.219512,0.000000,3.658537,"(Thriller, sum)"
5,8.196721,8.743169,11.475410,13.114754,6.010929,4.371585,0.0,9.836066,0.546448,3.278689,...,0.00000,14.754098,0.546448,5.464481,1.639344,6.010929,1.092896,0.000000,1.092896,"(Drama, sum)"


## Passo 3) Sistema de recomendação

O sistema de recomendação será baseado primeiramente no gênero que o usuário mais avalia, e depois no número de avaliações 5 estrelas dadas pelo universo de usuários.

O sistema seguirá os seguintes passos:

1. Filtrar lista de filmes que possuem o gênero de preferência do usuário e avaliação maior que 4.5.

2. Excluir os filmes que o usuário já assitiu

3. Fazer um ranking desta lista de filmes de acordo com o número de avaliações 5 estrelas

4. Sugerir os filmes no topo do ranking (1 ou mais)

In [11]:
# exemplo com um usuário:
user = 2
pref_genre = 'Sci-Fi'

# Filtrar lista de filmes que possuem o gênero definido e avaliação 5 estrelas
suggestion = ratings_movies[ratings_movies['genres'].str.contains(pref_genre)][ratings_movies['rating'] > 4.5]

# Excluir o que o usuário já assistiu
viewed = ratings_movies[ratings_movies['userId'] == user]['movieId']

for movie in viewed:
    suggestion = suggestion[suggestion['movieId'] != movie]

#Rankear pelo maior número de avaliações 5 estrelas e sugerir os melhores
n_suggestions = 5
final_suggestion = suggestion['title'].value_counts()[:n_suggestions]
final_suggestion

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Matrix, The (1999)                           18582
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)     9628
Aliens (1986)                                 7419
Clockwork Orange, A (1971)                    7388
Terminator, The (1984)                        7178
Name: title, dtype: int64

## Passo 3) Avaliação

In [12]:
# Verificando se o usuário assitiu à filmes indicados
ratings_movies[ratings_movies['userId'] == user][ratings_movies['title'] == 'Matrix, The (1999)']

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,userId,movieId,rating,title,genres
